In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, Model, Sequential

import numpy as np

from tqdm import tqdm

In [2]:
df = pd.read_csv("./dataset/career_pred.csv")

In [125]:
df.iloc[:49,:-1].columns

Index(['Acedamic percentage in Operating Systems', 'percentage in Algorithms',
       'Percentage in Programming Concepts',
       'Percentage in Software Engineering', 'Percentage in Computer Networks',
       'Percentage in Electronics Subjects',
       'Percentage in Computer Architecture', 'Percentage in Mathematics',
       'Percentage in Communication skills', 'Hours working per day',
       'Logical quotient rating', 'hackathons', 'coding skills rating',
       'public speaking points', 'can work long time before system?',
       'self-learning capability?', 'Extra-courses did', 'certifications',
       'workshops', 'talenttests taken?', 'olympiads',
       'reading and writing skills', 'memory capability score',
       'Interested subjects', 'interested career area ', 'Job/Higher Studies?',
       'Type of company want to settle in?',
       'Taken inputs from seniors or elders', 'interested in games',
       'Interested Type of Books', 'Salary Range Expected',
       'In a Rea

In [3]:
labels = df["Suggested Job Role"]
labels_unique = labels.unique().tolist()
labels = labels.map(lambda x: labels_unique.index(x))
labels = to_categorical(labels)

In [4]:
df = df[df.columns[:-1]]

In [ ]:
df[""]

In [218]:
k = {}
k["certification"]=df["certifications"].unique().tolist()
k["workshops"]=df["workshops"].unique().tolist()
k["Interested subjects"]=df["Interested subjects"].unique().tolist()
k["interested career area"]=df["interested career area "].unique().tolist()
k['Type of company want to settle in?']=df['Type of company want to settle in?'].unique().tolist()

In [219]:
k

{'certification': ['shell programming',
  'machine learning',
  'app development',
  'python',
  'r programming',
  'information security',
  'hadoop',
  'distro making',
  'full stack'],
 'workshops': ['cloud computing',
  'database security',
  'web technologies',
  'data science',
  'testing',
  'hacking',
  'game development',
  'system designing'],
 'Interested subjects': ['cloud computing',
  'networks',
  'hacking',
  'Computer Architecture',
  'programming',
  'parallel computing',
  'IOT',
  'data engineering',
  'Software Engineering',
  'Management'],
 'interested career area': ['system developer',
  'Business process analyst',
  'developer',
  'testing',
  'security',
  'cloud computing'],
 'Type of company want to settle in?': ['Web Services',
  'SAaS services',
  'Sales and Marketing',
  'Testing and Maintainance Services',
  'product development',
  'BPA',
  'Service Based',
  'Product based',
  'Cloud Services',
  'Finance']}

In [164]:
exclusion = [
    'Hours working per day',
    'Logical quotient rating',
    'hackathons',
    'coding skills rating',
    'public speaking points']

items_1 = [
    'Acedamic percentage in Operating Systems', 
    'percentage in Algorithms',
    'Percentage in Programming Concepts',
    'Percentage in Software Engineering', 
    'Percentage in Computer Networks',
    'Percentage in Electronics Subjects',
    'Percentage in Computer Architecture', 
    'Percentage in Mathematics',
    'Percentage in Communication skills']

items_2 = [
    'can work long time before system?',
    'self-learning capability?', 
    'Extra-courses did',
    'Management or Technical', 
    'Salary/work', 
    'hard/smart worker', 
    'worked in teams ever?', 
    'Introvert', 
    'Job/Higher Studies?']

items_3 = [
    'certifications',
    'workshops',
    'reading and writing skills',
    'memory capability score',
    'Interested subjects',
    'interested career area ',
    'Type of company want to settle in?']

print(items_1+items_2+items_3+exclusion)

def convert_to_x(row):
    global df

    labels = []

    for col in row.index:
        if col in items_1:
            labels.append(row[col]/100)
        elif col in items_2:
            labels.append(0 if row[col] == "no" else 1)
        elif col in items_3:
            uniques = df[col].unique().tolist()
            labels.append(uniques.index(row[col]))
        elif col in exclusion:
            labels.append(row[col])

    return labels

In [215]:
df.iloc[0].index

Index(['Acedamic percentage in Operating Systems', 'percentage in Algorithms',
       'Percentage in Programming Concepts',
       'Percentage in Software Engineering', 'Percentage in Computer Networks',
       'Percentage in Electronics Subjects',
       'Percentage in Computer Architecture', 'Percentage in Mathematics',
       'Percentage in Communication skills', 'Hours working per day',
       'Logical quotient rating', 'hackathons', 'coding skills rating',
       'public speaking points', 'can work long time before system?',
       'self-learning capability?', 'Extra-courses did', 'certifications',
       'workshops', 'talenttests taken?', 'olympiads',
       'reading and writing skills', 'memory capability score',
       'Interested subjects', 'interested career area ', 'Job/Higher Studies?',
       'Type of company want to settle in?',
       'Taken inputs from seniors or elders', 'interested in games',
       'Interested Type of Books', 'Salary Range Expected',
       'In a Rea

In [165]:
xs = np.array([convert_to_x(df.iloc[i]) for i in tqdm(df.index)])
np.save("./dataset/xs.npy", xs)

100%|██████████| 20000/20000 [03:20<00:00, 99.98it/s] 


In [6]:
xs = np.load("./dataset/xs.npy")

In [179]:
def group_list(l, group_size):
    for i in range(0, len(l), group_size):
        yield l[i:i+group_size]

BATCH_SIZE = 100

split_at = int(len(xs)//BATCH_SIZE * 0.8)
dataset = tf.data.Dataset.from_tensor_slices((xs, labels)).shuffle(1000).batch(BATCH_SIZE)

train_dataset = dataset.take(split_at)
test_dataset = dataset.skip(split_at)

In [167]:
sum_items = len(items_1) + len(items_2) + len(items_3) + len(exclusion)

In [213]:
def make_model():
    _input = layers.Input(shape=(sum_items,))

    a, b, c, d = tf.split(_input, [len(items_1), len(items_2), len(items_3), len(exclusion)], axis=1)

    e1 = layers.Dense(32, activation="relu")(a)
    # # e1 = layers.LeakyReLU(0.2)(e1)
    e2 = layers.Dense(32, activation="relu")(b)
    # # e1 = layers.LeakyReLU(0.2)(e1)
    e3 = layers.Dense(32, activation="relu")(c)
    # # e1 = layers.LeakyReLU(0.2)(e1)
    e4 = layers.Dense(32, activation="relu")(d)
    # e4 = layers.LeakyReLU(0.2)(e4)

    e = layers.concatenate([e1, e2, e3, e4])
    # e = layers.Dropout(0.2)(e)
    # e = layers.LayerNormalization()(e)

    # use multihead attention
    attention = layers.MultiHeadAttention(num_heads=8, key_dim=8, attention_axes=(1,))(_input, _input)
    # attention = layers.LayerNormalization()(attention)

    e = layers.Dense(1024, activation="relu")(e)
    e = layers.Dropout(0.2)(e)
    e = layers.Dense(1024, activation="relu")(e)
    e = layers.Dropout(0.2)(e)
    e = layers.Dense(512, activation="relu")(e)
    e = layers.Dropout(0.2)(e)
    e = layers.Dense(256, activation="relu")(e)
    e = layers.Dense(labels.shape[-1], activation="softmax")(e)

    return Model(inputs=[_input], outputs=e)

model = make_model()
model.compile(
    # optimizer=keras.optimizers.Adam(learning_rate=0.0001, epsilon=1e-07),
    optimizer=keras.optimizers.Adagrad(learning_rate=0.01),
    # optimizer=keras.optimizers.Nadam(learning_rate=0.0001),
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy"]
)

In [216]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=1000,
)

Epoch 1/1000
160/160 [==============================] - 2s 15ms/step - loss: 0.3903 - categorical_accuracy: 0.8844 - val_loss: 5.4808 - val_categorical_accuracy: 0.2280
Epoch 2/1000
160/160 [==============================] - 2s 10ms/step - loss: 0.3783 - categorical_accuracy: 0.8898 - val_loss: 5.4986 - val_categorical_accuracy: 0.2278_accuracy: 0.
Epoch 3/1000
160/160 [==============================] - 2s 11ms/step - loss: 0.3769 - categorical_accuracy: 0.8873 - val_loss: 5.5024 - val_categorical_accuracy: 0.2253
Epoch 4/1000
160/160 [==============================] - 2s 10ms/step - loss: 0.3841 - categorical_accuracy: 0.8843 - val_loss: 5.5069 - val_categorical_accuracy: 0.2295
Epoch 5/1000
160/160 [==============================] - 1s 9ms/step - loss: 0.3753 - categorical_accuracy: 0.8889 - val_loss: 5.5155 - val_categorical_accuracy: 0.2262
Epoch 6/1000
160/160 [==============================] - 2s 11ms/step - loss: 0.3822 - categorical_accuracy: 0.8857 - val_loss: 5.5007 - val_cat

KeyboardInterrupt: 

In [157]:
x = [
    0.82,0.81,0.85,0.78,0.80,0.70,0.80,0.91,0.81,
    10,8,3,8,6,
    1,1,1,
    1,3,0,1,
    2,2,7,2,
    0,8,0,1,29,0,
    1,1,1,1,1,1,1
]

In [158]:
(model(np.expand_dims(np.array(x), axis=0)).numpy() * 100)[0].tolist()

[9.510838508605957,
 0.009297593496739864,
 0.006974688731133938,
 0.07236986607313156,
 1.9401112794876099,
 0.9470861554145813,
 4.636445045471191,
 6.032196521759033,
 0.14621862769126892,
 0.1669691503047943,
 13.264496803283691,
 0.17827068269252777,
 0.0003387811593711376,
 0.08490696549415588,
 0.12952132523059845,
 0.000183049138286151,
 0.6904901266098022,
 9.815388693823479e-06,
 33.59199523925781,
 0.026268435642123222,
 0.9199110865592957,
 0.0011916409712284803,
 0.06737217307090759,
 14.64106273651123,
 10.616471290588379,
 0.0017655928386375308,
 0.03384695202112198,
 0.0004062606021761894,
 0.00013492576545104384,
 1.534615397453308,
 0.003360724775120616,
 0.4115973114967346,
 0.2522604465484619,
 0.08101014792919159]

In [143]:
labels_unique[0]

'Database Developer'

In [144]:
test = np.expand_dims(np.array(convert_to_x(df.iloc[1000])), axis=0)
labels[1000].tolist().index(1)

15

In [145]:
np.argmax(model(test))

15

In [97]:
df["interested career area "].unique()

array(['system developer', 'Business process analyst', 'developer',
       'testing', 'security', 'cloud computing'], dtype=object)

In [154]:
labels_unique

['Database Developer',
 'Portal Administrator',
 'Systems Security Administrator',
 'Business Systems Analyst',
 'Software Systems Engineer',
 'Business Intelligence Analyst',
 'CRM Technical Developer',
 'Mobile Applications Developer',
 'UX Designer',
 'Quality Assurance Associate',
 'Web Developer',
 'Information Security Analyst',
 'CRM Business Analyst',
 'Technical Support',
 'Project Manager',
 'Information Technology Manager',
 'Programmer Analyst',
 'Design & UX',
 'Solutions Architect',
 'Systems Analyst',
 'Network Security Administrator',
 'Data Architect',
 'Software Developer',
 'E-Commerce Analyst',
 'Technical Services/Help Desk/Tech Support',
 'Information Technology Auditor',
 'Database Manager',
 'Applications Developer',
 'Database Administrator',
 'Network Engineer',
 'Software Engineer',
 'Technical Engineer',
 'Network Security Engineer',
 'Software Quality Assurance (QA) / Testing']